# Main

In [1]:
import os
import json
import time
import requests
import pandas as pd

from datetime import datetime

In [33]:
# Temporary setup
pd.set_option('display.max_columns', 100)

In [2]:
date_time = datetime.now()
                    
if time.tzname[0] == 'UTC':
    date_time = date_time - timedelta(hours=3)

In [3]:
datetime = date_time.strftime("%Y%m%dT%H:%M:%S")

In [4]:
timestamp = int(date_time.timestamp())

In [5]:
formated_datetime = date_time.strftime("%Y%m%d_%Hh%Mm%Ss")

In [6]:
users = {
    "viikset": "BR1",
    "TADALA": "TADAL",
    "DIGAOTJS": "BR1",
    "mBirth": "BR1",
    "Kubata": "4881"
}

In [7]:
puuids = {
    "viikset": "f2f7c867-27ba-5291-a23c-bbdcfc079e76",
    "TADALA": "35a2368e-586f-53ec-8ee8-ea9ebdca7d6f",
    "DIGAOTJS": "2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663",
    "mBirth": "6c241ab0-1c2f-5d5b-8908-4e809df8f6ca",
    "Kubata": "b2310858-1741-5d6f-8499-cb1f85bd182a"
}

## Get Account Data

In [8]:
all_match_ids = []

df_account = pd.DataFrame()
df_mmr_data = pd.DataFrame()
df_mmr_history = pd.DataFrame()
df_matchs = pd.DataFrame()


for name, tag in users.items():
    
    print(f'Get data for {name}...')
    
    # Accounts Data
    #print(f'Get account data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}")
    
    with open(f'account/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_account = pd.concat([df_account, pd.json_normalize(response.json()['data'])])
        
        
    # MMR Data
    #print(f'Get MMR data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr/na/{name}/{tag}")
    
    with open(f'mmr_data/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_mmr_data = pd.concat([df_mmr_data, pd.json_normalize(response.json()['data'])])  
        
    # MMR History
    #print(f'Get MMR History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr-history/na/{name}/{tag}")
    
    with open(f'mmr_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    temp_mmr_history = pd.json_normalize(response.json()['data'])
    temp_mmr_history['name'] = response.json()['name']
    temp_mmr_history['tag'] = response.json()['tag']
    
    df_mmr_history = pd.concat([df_mmr_history, temp_mmr_history])
    
    # Match History
    #print(f'Get Match History data for {name}...')
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/matches/na/{name}/{tag}?filter=competitive")
    
    with open(f'match_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    # Listing All Match IDs
    metadata = []
    for i in response.json()['data']:
        all_match_ids.append(i['metadata']['matchid'])
        
        metadata.append(i['metadata'])
        
    df_matchs = pd.concat([df_matchs, pd.DataFrame(metadata)])
    
    print('\n')

df_account['datetime'] = datetime
df_mmr_data['datetime'] = datetime

Get data for viikset...
Get account data for viikset...
Get MMR data for viikset...
Get MMR History data for viikset...
Get Match History data for viikset...


Get data for TADALA...
Get account data for TADALA...
Get MMR data for TADALA...
Get MMR History data for TADALA...
Get Match History data for TADALA...


Get data for DIGAOTJS...
Get account data for DIGAOTJS...
Get MMR data for DIGAOTJS...
Get MMR History data for DIGAOTJS...
Get Match History data for DIGAOTJS...


Get data for mBirth...
Get account data for mBirth...
Get MMR data for mBirth...
Get MMR History data for mBirth...
Get Match History data for mBirth...


Get data for Kubata...
Get account data for Kubata...
Get MMR data for Kubata...
Get MMR History data for Kubata...
Get Match History data for Kubata...




# Get All Match History

In [9]:
all_match_ids = list(set(all_match_ids))

In [38]:
metadata = []
rounds = []
kills = []

# Players Dataframes
session_playtime = pd.DataFrame()
behavior = pd.DataFrame()
platform = pd.DataFrame()
ability_casts = pd.DataFrame()
assets = pd.DataFrame()
stats = pd.DataFrame()
economy = pd.DataFrame()
player = pd.DataFrame()


df_players = pd.DataFrame()
df_teams = pd.DataFrame()

for matchid in all_match_ids:
    
    # Request
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/match/{matchid}")
    
    # Save file
    with open(f'match/{matchid}.json', 'w') as f:
        json.dump(response.json(), f)
        
    match = response.json()['data']
        
    # Match Metadata    
    metadata.append(match['metadata'])
    
    # Match Players
    json_players = match['players']['all_players'].copy()

    for player in json_players:

        puuid = player['puuid']

        temp_session_playtime = player.pop('session_playtime')
        temp_session_playtime = pd.json_normalize(temp_session_playtime)
        temp_session_playtime['matchid'] = matchid
        temp_session_playtime['puuid'] = puuid
        session_playtime = pd.concat([session_playtime, temp_session_playtime])

        temp_behavior = player.pop('behavior') 
        temp_behavior = pd.json_normalize(temp_behavior)
        temp_behavior['matchid'] = matchid
        temp_behavior['puuid'] = puuid
        behavior = pd.concat([behavior, temp_behavior])

        temp_platform = player.pop('platform')
        temp_platform = pd.json_normalize(temp_platform)
        temp_platform['matchid'] = matchid
        temp_platform['puuid'] = puuid
        platform = pd.concat([platform, temp_platform])

        temp_ability_casts = player.pop('ability_casts')
        temp_ability_casts = pd.json_normalize(temp_ability_casts)
        temp_ability_casts['matchid'] = matchid
        temp_ability_casts['puuid'] = puuid
        ability_casts = pd.concat([ability_casts, temp_ability_casts])
        
        temp_assets = player.pop('assets')
        temp_assets = pd.json_normalize(temp_assets)
        temp_assets['matchid'] = matchid
        temp_assets['puuid'] = puuid
        assets = pd.concat([assets, temp_assets])

        temp_stats = player.pop('stats')
        temp_stats = pd.json_normalize(statstemp_stats
        temp_stats['matchid'] = matchid
        temp_stats['puuid'] = puuid
        stats = pd.concat([stats, temp_stats])

        temp_economy = player.pop('economy')
        temp_economy = pd.json_normalize(temp_economy)
        temp_economy['matchid'] = matchid
        temp_economy['puuid'] = puuid
        economy = pd.concat([economy, temp_economy])
                                       
        temp_player = pd.json_normalize(pltemp_playerayer)
        temp_player['matchid'] = matchid
        player = pd.concat([player, temp_player])
    
    # Match Teams
    temp_teams = pd.json_normalize(match['teams'])
    temp_teams['matchid'] = matchid
    df_teams = pd.concat([df_teams, temp_teams])
    
    
    rounds.append(match['rounds'])
    
    
    kills.append(match['kills'])
        
    #break
    
df_metadata = pd.DataFrame(metadata)


In [249]:
match = response.json()['data']

json_rounds = match['rounds'].copy()

plant_events = pd.DataFrame()

count = 1

for row_round in json_rounds:
    
    # Plant Events
    plant_events = row_round.pop('plant_events')
    player_locations_on_plant = plant_events.pop('player_locations_on_plant')
    
    plant_events = pd.json_normalize(plant_events)
    plant_events['matchid'] = matchid
    plant_events['round'] = count
    
    player_locations_on_plant = pd.json_normalize(player_locations_on_plant)
    player_locations_on_plant['matchid'] = matchid
    player_locations_on_plant['round'] = count
    
    # Defuse Events
    defuse_events = row_round.pop('defuse_events')
    player_locations_on_defuse = defuse_events.pop('player_locations_on_defuse')
    
    defuse_events = pd.json_normalize(defuse_events)
    defuse_events['matchid'] = matchid
    defuse_events['round'] = count

    player_locations_on_defuse = pd.json_normalize(player_locations_on_defuse)
    player_locations_on_defuse['matchid'] = matchid
    player_locations_on_defuse['round'] = count
    
    # Player Stats
    player_stats = row_round.pop('player_stats')
    
    for plyr in player_stats:
        
    
    count += 1
    break
    
    

In [264]:
for plyr in player_stats:
        break

In [265]:
plyr.keys()

dict_keys(['ability_casts', 'player_puuid', 'player_display_name', 'player_team', 'damage_events', 'damage', 'bodyshots', 'headshots', 'legshots', 'kill_events', 'kills', 'score', 'economy', 'was_afk', 'was_penalized', 'stayed_in_spawn'])

In [283]:
plyr['stayed_in_spawn']

False

In [284]:
pd.json_normalize(plyr)

,player_puuid,player_display_name,player_team,damage_events,damage,bodyshots,headshots,legshots,kill_events,kills,score,was_afk,was_penalized,stayed_in_spawn,ability_casts.c_casts,ability_casts.q_casts,ability_casts.e_cast,ability_casts.x_cast,economy.loadout_value,economy.weapon.id,economy.weapon.name,economy.weapon.assets.display_icon,economy.weapon.assets.killfeed_icon,economy.armor.id,economy.armor.name,economy.armor.assets.display_icon,economy.remaining,economy.spent
0,35a2368e-586f-53ec-8ee8-ea9ebdca7d6f,TADALA#TADAL,Red,[{'receiver_puuid': '9262f939-87a5-5974-9478-4...,30,1,0,0,[],0,55,False,False,False,None,None,None,None,800,1BAA85B4-4C70-1284-64BB-6481DFC3BB4E,Ghost,https://media.valorant-api.com/weapons/1baa85b...,https://media.valorant-api.com/weapons/1baa85b...,None,None,None,0,800
